<a href="https://colab.research.google.com/github/EnDFLab/EMCS_Project/blob/smart_meter_data_analytics/holidays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nepali-datetime
import pandas as pd
import numpy as np
import datetime
import nepali_datetime as ndt

#print(ndt.datetime.now())

#print(ndt.date.from_datetime_date(datetime.date(1998, 9, 14))) #AD to BS

#url = "https://drive.google.com/uc?export=download&id=1mtUtj8iJTyUSeBrM0DMI1X0zuqS9QidB"


In [ ]:
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1nAQbt22VEWJV_q4Aa4aaViE9putEiPZl")
df.dtypes
df["Date"] = pd.to_datetime(df['Date'])
#df.dtypes

#print(df.head())
#print(df.Date[0])

all_dates_2024 = pd.date_range(start="2024-01-01", end="2024-12-31", freq="15min")
df.set_index("Date", inplace=True)
df = df.reindex(all_dates_2024)
df.index.name = "Date"
df = df.reset_index()

#print(df.head(13))
#ndt.date.from_datetime_date() expects a datetime.date object so expicityly convert Date into datetime object

df["nepali_date"] = df["Date"].apply(lambda x: ndt.date.from_datetime_date(x.date()))

date = df.pop('nepali_date')

df.insert(1, 'Date (B.S)', date)

df["Day"] = df.Date.dt.day
day = df.pop("Day")
df.insert(2,"Day",day)

#columns_to_fill = ['SoS', 'SoE']
#df[columns_to_fill] = df[columns_to_fill].fillna(0).astype(int)
#df[columns_to_fill] = df[columns_to_fill].fillna(0).astype(int)
#df["both"] = df["SoS"] & df["SoE"]

df['Month'] = df['Date'].dt.month

#df = df.dropna()
df['Holiday_name'] = df.groupby(['Month','Day'])[['Holiday_name']].transform(lambda x: x.bfill().ffill())
df['SoS'] = df.groupby(['Month','Day'])['SoS'].transform(lambda x: x.bfill().ffill())
df['SoE'] = df.groupby(['Month','Day'])['SoE'].transform(lambda x: x.bfill().ffill())

columns_to_fill = ['SoS', 'SoE']
df[columns_to_fill] = df[columns_to_fill].fillna(0).astype(int)
df["both"] = df["SoS"] & df["SoE"]

df = df.dropna()
df.drop(columns=["Month"], inplace=True)

df.set_index("Date", inplace=True)
df.to_csv("Holidays_exported.csv")

print(df.head())




<ipython-input-50-2c732bea04fa>:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Holiday_name'] = df.groupby(['Month','Day'])[['Holiday_name']].transform(lambda x: x.bfill().ffill())


                     Date (B.S)  Day     Holiday_name  SoS  SoE  both
Date                                                                 
2024-01-12 00:00:00  2080-09-27   12  Prithvi Jayanti    1    1     1
2024-01-12 00:15:00  2080-09-27   12  Prithvi Jayanti    1    1     1
2024-01-12 00:30:00  2080-09-27   12  Prithvi Jayanti    1    1     1
2024-01-12 00:45:00  2080-09-27   12  Prithvi Jayanti    1    1     1
2024-01-12 01:00:00  2080-09-27   12  Prithvi Jayanti    1    1     1


<ipython-input-50-2c732bea04fa>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["Month"], inplace=True)
